In [ ]:
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
import torch
method = 'cell2fate'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
dataset = 'MouseErythroid'
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((20, 40, 10))
# Model params:
mod_list = [0.5, 1, 1.5, 2.0]

def robust_optimization(mod, save_dir, max_epochs = [200, 400], lr = [0.01, 0.01], **model_kwargs):
    n_modules = mod.module.model.n_modules
    adata = mod.adata
    print('First optimization run.')
    mod.train(use_gpu=True, max_epochs = max_epochs[0], lr = lr[0])
    sample_kwarg = {"num_samples": 1, "batch_size" : 1000,
                     "use_gpu" : True, 'return_samples': False}
    mod.adata = mod.export_posterior(mod.adata, sample_kwargs=sample_kwarg)
    t_c = np.argsort(np.array(mod.samples['post_sample_means']['t_c']).flatten())/len(np.array(mod.samples['post_sample_means']['t_c']))
    t_c_reversed = -1*(t_c - np.max(t_c))
    print('Second optimization run.')
    del mod
    mod1 = c2f.Cell2fate_DynamicalModel_PreprocessedCounts(adata, n_modules = n_modules,
                                        init_vals = {'t_c': torch.tensor(t_c).reshape([len(t_c), 1, 1])},
                                       **model_kwargs)
    mod1.train(use_gpu=True, max_epochs = max_epochs[1], lr = lr[1])
    history1 = mod1.history
    mod1.save(save_dir+'c2f_model', overwrite=True)
    mod1.adata.write(save_dir+"c2f_model_anndata.h5ad")
    del mod1
    print('Third optimization run.')
    mod2 = c2f.Cell2fate_DynamicalModel_PreprocessedCounts(adata, n_modules = n_modules,
                                        init_vals = {'t_c': torch.tensor(t_c_reversed).reshape([len(t_c_reversed), 1, 1])},
                                        **model_kwargs)
    del adata
    mod2.train(use_gpu=True, max_epochs = max_epochs[1], lr = lr[1])
    history2 = mod2.history

    iter_start=0
    iter_end=-1

    fig, ax = plt.subplots(1,2, figsize = (15,5))

    iter_end = len(history1["elbo_train"])

    ax[0].plot(
        history1["elbo_train"].index[iter_start:iter_end],
        np.array(history1["elbo_train"].values.flatten())[iter_start:iter_end],
        label="Original Direction",
    )
    ax[0].plot(
        history2["elbo_train"].index[iter_start:iter_end],
        np.array(history2["elbo_train"].values.flatten())[iter_start:iter_end],
        label="Reversed Direction",
    )
    ax[0].legend()
    ax[0].set_xlim(0, len(history1["elbo_train"]))
    ax[0].set_xlabel("Training epochs")
    ax[0].set_ylabel("-ELBO loss")

    ax[1].plot(
        history1["elbo_train"].index[iter_start:iter_end],
        np.array(history1["elbo_train"].values.flatten())[iter_start:iter_end],
        label="Original Direction",
    )
    ax[1].plot(
        history2["elbo_train"].index[iter_start:iter_end],
        np.array(history2["elbo_train"].values.flatten())[iter_start:iter_end],
        label="Reversed Direction",
    )
    ax[1].legend()
    ax[1].set_xlim(np.round(0.8*len(history1["elbo_train"])), len(history1["elbo_train"]))
    ax[1].set_xlabel("Training epochs")
    ax[1].set_ylabel("-ELBO loss")
    plt.tight_layout()
    plt.show()
    
    print('-ELBO second run:', np.mean(np.array(history1['elbo_train'][-50:])))
    print('-ELBO third run:', np.mean(np.array(history2['elbo_train'][-50:])))
    
    if np.mean(np.array(history1['elbo_train'][-50:])) > np.mean(np.array(history2['elbo_train'][-50:])):
        return mod2
    else:
        del mod2
        adata = sc.read_h5ad(save_dir+"c2f_model_anndata.h5ad")
        mod1 = c2f.Cell2fate_DynamicalModel_PreprocessedCounts.load(save_dir+'c2f_model', adata)
        return mod1 

for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in range(len(mod_list)):
                print(i)
                print(j)
                print(k)
                n_genes = n_genes_list[i]
                min_counts = n_counts_list[j]
                model_index = str(i) + '-' + str(j) + '-' + str(k)
                save_name = method + '_' + dataset
                if exists(save_dir + save_name + '_CBDC_And_Posterior_withBatch_preprocessed_oneRun.csv'):
                    tab = pd.read_csv(save_dir + save_name + '_CBDC_And_Posterior_withBatch_preprocessed_oneRun.csv', index_col = 0)
                    if model_index in tab.index:
                        continue
                adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
                scv.pp.filter_and_normalize(adata, min_shared_counts=min_counts, n_top_genes=n_genes)
                scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
                c2f.Cell2fate_DynamicalModel_PreprocessedCounts.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced',
                                                          batch_key = 'sequencing.batch')    
                n_modules = int(np.round(c2f.utils.get_max_modules(adata)*mod_list[k]))
                mod = c2f.Cell2fate_DynamicalModel_PreprocessedCounts(adata,
                                                   n_modules = n_modules)   
                mod.train()
                sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
                     "use_gpu" : True, 'return_samples': False}
                adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
                mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
                # Calculate performance metrics:
                file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
                ground_truth = pickle.load(file)
                metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
                cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                            for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
                if exists(save_dir + save_name + '_CBDC_And_Posterior_withBatch_preprocessed_oneRun.csv'):
                    tab = pd.read_csv(save_dir + save_name + '_CBDC_And_Posterior_withBatch_preprocessed_oneRun.csv', index_col = 0)
                else:
                    c_names = ['CBDC']
                    tab = pd.DataFrame(columns = c_names)
                tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
                tab.loc[model_index, 'splicing_alpha'] = mod.samples['post_sample_means']['splicing_alpha']
                tab.loc[model_index, 'splicing_mean'] = mod.samples['post_sample_means']['splicing_mean']
                tab.loc[model_index, 'beta_g_mean'] = np.mean(mod.samples['post_sample_means']['beta_g'])
                tab.loc[model_index, 'beta_g_var'] = np.var(mod.samples['post_sample_means']['beta_g'])
                tab.loc[model_index, 'degredation_alpha'] = mod.samples['post_sample_means']['degredation_alpha']
                tab.loc[model_index, 'degredation_mean'] = mod.samples['post_sample_means']['degredation_mean']
                tab.loc[model_index, 'gamma_g_mean'] = np.mean(mod.samples['post_sample_means']['gamma_g'])
                tab.loc[model_index, 'gamma_g_var'] = np.var(mod.samples['post_sample_means']['gamma_g'])
                tab.loc[model_index, 'factor_level_g_mean'] = np.mean(mod.samples['post_sample_means']['factor_level_g'])
                tab.loc[model_index, 'factor_level_g_var'] = np.var(mod.samples['post_sample_means']['factor_level_g'])
                tab.loc[model_index, 'g_fg_mean'] = np.mean(mod.samples['post_sample_means']['g_fg'])
                tab.loc[model_index, 'g_fg_var'] = np.var(mod.samples['post_sample_means']['g_fg'])
                tab.loc[model_index, 'A_mgON_mean'] = np.mean(mod.samples['post_sample_means']['A_mgON'])
                tab.loc[model_index, 'A_mgON_var'] = np.var(mod.samples['post_sample_means']['A_mgON'])
                tab.loc[model_index, 'lam_mu'] = mod.samples['post_sample_means']['lam_mu']
                tab.loc[model_index, 'lam_sd'] = mod.samples['post_sample_means']['lam_sd']
                tab.loc[model_index, 'lam_m_mu_mean'] = np.mean(mod.samples['post_sample_means']['lam_m_mu'])
                tab.loc[model_index, 'lam_m_mu_var'] = np.var(mod.samples['post_sample_means']['lam_m_mu'])
                tab.loc[model_index, 'lam_mi_mean'] = np.mean(mod.samples['post_sample_means']['lam_mi'])
                tab.loc[model_index, 'lam_mi_var'] = np.var(mod.samples['post_sample_means']['lam_mi'])
                tab.loc[model_index, 'Tmax'] = mod.samples['post_sample_means']['Tmax']
                tab.loc[model_index, 't_c_loc'] = mod.samples['post_sample_means']['t_c_loc']
                tab.loc[model_index, 't_c_scale'] = mod.samples['post_sample_means']['t_c_scale']
                tab.loc[model_index, 'T_c_mean'] = np.mean(mod.samples['post_sample_means']['T_c'])
                tab.loc[model_index, 'T_c_var'] = np.var(mod.samples['post_sample_means']['T_c'])
                tab.loc[model_index, 't_delta_mean'] = np.mean(mod.samples['post_sample_means']['t_delta'])
                tab.loc[model_index, 't_delta_var'] = np.var(mod.samples['post_sample_means']['t_delta'])
                tab.loc[model_index, 'T_mON_mean'] = np.mean(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_var'] = np.var(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_mean'] = np.mean(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_var'] = np.var(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mOFF_mean'] = np.mean(mod.samples['post_sample_means']['T_mOFF'])
                tab.loc[model_index, 'T_mOFF_var'] = np.var(mod.samples['post_sample_means']['T_mOFF'])
                tab.loc[model_index, 'mu_expression_mean'] = np.mean(mod.samples['post_sample_means']['mu_expression'])
                tab.loc[model_index, 'mu_expression_var'] = np.var(mod.samples['post_sample_means']['mu_expression'])
                tab.to_csv(save_dir + save_name + '_CBDC_And_Posterior_withBatch_preprocessed_oneRun.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-18 12:28:37
0
0
0
Filtered out 47456 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:03:36) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Leiden clustering ...
Number of Leiden Clusters: 9
Maximal Number of Modules: 10


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 500/500: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [11:19<00:00,  1.36s/it, v_num=1, elbo_train=1.69e+7]
sample_kwargs['batch_size'] 9815
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.23it/s]
Computing total RNAvelocity ...


  0%|          | 0/9815 [00:00<?, ?cells/s]

...

Number of Leiden Clusters: 9
Maximal Number of Modules: 10
Epoch 500/500: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [22:08<00:00,  2.66s/it, v_num=1, elbo_train=2.33e+7]
sample_kwargs['batch_size'] 9815
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.03it/s]
Computing total RNAvelocity ...


  0%|          | 0/9815 [00:00<?, ?cells/s]

# Cross-Boundary Direction Correctness (A->B)
{('Blood progenitors 1', 'Blood progenitors 2'): 0.4453066757820847, ('Blood progenitors 2', 'Erythroid1'): 0.557590618951337, ('Erythroid1', 'Erythroid2'): 0.6495025410231053, ('Erythroid2', 'Erythroid3'): 0.7091302589515536}
Total Mean: 0.5903825236770202
# In-cluster Coherence
{'Blood progenitors 1': 0.6871128, 'Blood progenitors 2': 0.7676731, 'Erythroid1': 0.74039173, 'Erythroid2': 0.68774444, 'Erythroid3': 0.73996717}
Total Mean: 0.724577784538269
1
0
3
Filtered out 47456 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 3000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:04) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:02) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Leiden clustering ...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 9
Maximal Number of Modules: 10
Epoch 197/500:  39%|████████████████████████████████████████████████████████████▎                                                                                             | 196/500 [10:40<17:11,  3.39s/it, v_num=1, elbo_train=2.34e+7]

In [ ]:
tab = pd.read_csv('/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/cell2fate_MouseErythroid_CBDC_And_Posterior_withBatch_preprocessed.csv')

In [ ]:
tab